In [2]:
import os
from Bio import SeqIO, Seq
import pandas as pd

In [3]:
# read the xlsx file and create DataFrame
uni_file = 'uniprot-filtered-organism__Bacillus+subtilis+(strain+168)+[224308]%2--.xlsx'
assert(os.path.exists(uni_file)) #  sanity check
df_prot = pd.read_excel(uni_file)

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# print the DF head
print('df:\n', df_prot.head())

df:
     Entry  Entry name    Status  \
0  P94485  YNAG_BACSU  reviewed   
1  O32014  YQZH_BACSU  reviewed   
2  O34838  YOPW_BACSU  reviewed   
3  O34460  YTRI_BACSU  reviewed   
4  O34702  YOTC_BACSU  reviewed   

                                       Protein names          Gene names  \
0                       Uncharacterized protein YnaG       ynaG BSU17550   
1                       Uncharacterized protein YqzH       yqzH BSU23720   
2  SPbeta prophage-derived uncharacterized protei...       yopW BSU20740   
3                  Sporulation membrane protein YtrI       ytrI BSU29240   
4  SPbeta prophage-derived uncharacterized protei...  yotC yokC BSU19930   

                         Organism  Length  \
0  Bacillus subtilis (strain 168)    91.0   
1  Bacillus subtilis (strain 168)    68.0   
2  Bacillus subtilis (strain 168)   111.0   
3  Bacillus subtilis (strain 168)   167.0   
4  Bacillus subtilis (strain 168)    63.0   

                                       Transmembrane  \


In [158]:
# find all the protiens' gene name
prot_names = [x.split(" ")[0] for x in df_prot['Gene names'].apply(str)]
print(prot_names[0:5])

['ynaG', 'yqzH', 'yopW', 'ytrI', 'yotC']


In [95]:
## read file and import details
gene_bank_file = 'BS168.gb'
 # making sure that the path is valid
assert(os.path.exists(gene_bank_file))

with open(gene_bank_file, "r") as input_handle:
    record_gb = SeqIO.read(input_handle, "genbank")

features = record_gb.features
print(type(features))

print(len(features))

<class 'list'>
9074


In [144]:
# find all CDS in GeneBank file
features_prot = []

for feature in features:
    if feature.type == 'CDS' and feature.qualifiers.get('gene') is not None:
        features_prot.append(feature.qualifiers.get('gene')[0])

print(features_prot[0:10])
print(len(features_prot))

['dnaA', 'dnaN', 'rlbA', 'recF', 'remB', 'gyrB', 'gyrA', 'yaaC', 'guaB', 'dacA']
4177


In [152]:
# compare lists and create 3 lists

# same protien in both
same_prot = set(features_prot).intersection(set(prot_names))
# protien exist only in genebank file 
gb_only = set(features_prot).difference(set(prot_names))
# protien exist only in Uniport file 
uni_only = set(prot_names).difference(set(features_prot))

print('Same protiens in both files: ', len(same_prot))
print('Protiens in GeneBank only: ', len(gb_only))
print('Protiens in UniProt only: ', len(uni_only))  

# print(list(same_prot)[0:5])

Same protiens in both files:  3715
Protiens in GeneBank only:  456
Protiens in UniProt only:  3224


check where is the difference come from

In [169]:
# check protien in GB  
gb_same = []
gb_diff = []

for feature in features:
    if feature.type == 'CDS':
        if feature.qualifiers.get('gene') is not None:
            if feature.qualifiers.get('gene')[0] in list(same_prot):
                gb_same.append(feature)
            else:
                gb_diff.append(feature)

print(len(gb_same), len(gb_diff))

3721 456


try to get datafram from the original DF so the name is in the gene name list "prot name" above.

In [167]:
# df_diff = df_prot.loc[df_prot.apply(lambda x: x['Gene names'].apply(str).split(" ")[0] in list(uni_only))]


KeyError: 'Gene names'